In [30]:
import gem5art.artifact
import zipfile
import os
from uuid import UUID

## Connect to the database and retrieve the result files

In [2]:
conn = gem5art.artifact.getDBConnection()

results_it = gem5art.artifact.getByName('results')

for result in results_it:
    if "spec-experiments" in result.path:
        filename = "zip"
        conn.downloadFile(result._id, filename)
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall(os.getcwd())
        os.remove(filename)

## Constants / Util functions

In [21]:
# Copied from launch_spec2006_experiments.py
cpus = ['kvm', 'atomic', 'o3', 'timing']
benchmark_sizes = {'kvm':    ['test', 'ref'],
                    'atomic': ['test'],
                    'o3':     ['test'],
                    'timing': ['test']
                   }
benchmarks = ['401.bzip2','403.gcc','410.bwaves','416.gamess','429.mcf',
                  '433.milc','434.zeusmp','435.gromacs','436.cactusADM',
                  '437.leslie3d','444.namd','445.gobmk','453.povray',
                  '454.calculix','456.hmmer','458.sjeng','459.GemsFDTD',
                  '462.libquantum','464.h264ref','465.tonto','470.lbm',
                  '471.omnetpp','473.astar','481.wrf','482.sphinx3',
                  '998.specrand','999.specrand']

# CPU display names
cpu_name_of = { 'kvm': 'KVM',
                'o3': 'O3CPU',
                'atomic': 'AtomicCPU',
                'timing': 'TimingSimpleCPU'
              }

# Util functions
# https://stackoverflow.com/questions/5676646/how-can-i-fill-out-a-python-string-with-spaces
right_padded_str = lambda s, p: '{message:{fill}{align}{width}}'.format(
   message = s,
   fill = ' ',
   align = '<',
   width = p,
)

## Verifying SPEC2006 Runs

### Checking SPEC2006 logs
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
The results folder of each benchmark has a folder named `speclogs`, which contains the logs of the run spec commands. 
There are two logs in this folder: `CPU2006.001.log` and `CPU2006.002.log`. 
The former is the log of compiling SPEC benchmarks, which is generated when we compile SPEC benchmarks while we create the disk image. 
The latter is the log of the benchmark run. 
So, we only interest in `CPU2006.002.log`.  

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
If the benchmark run is successful, there will be a line starting with `Success: 1x` followed by `benchmark_name`. 
We will look for this line in each `CPU2006.002.log` file.

In [22]:
# Results folder structure: results/cpu/benchmark_size/benchmark_name
# A simple `grep` would extract all necessary information, but it is not a cross-platform solution.

speclogs_verifying = {}

for cpu in cpus:
    speclogs_verifying[cpu] = {}
    for benchmark_size in benchmark_sizes[cpu]:
        speclogs_verifying[cpu][benchmark_size] = {}
        for benchmark_name in benchmarks:
            speclogs_verifying[cpu][benchmark_size][benchmark_name] = ""
            results_folder_path = "results/{}/{}/{}/".format(cpu, benchmark_size, benchmark_name)
            # check if the results folder exists
            if not os.path.isdir(results_folder_path):
                speclogs_verifying[cpu][benchmark_size][benchmark_name] = "No results folder"
                continue
            # check SPEC log
            try:
                with open("{}/speclogs/CPU2006.002.log".format(results_folder_path), "r") as f:
                    result = f.read().find('Success: 1x'.format(benchmark_name))
                    if result > 0:
                        speclogs_verifying[cpu][benchmark_size][benchmark_name] = "Success"
                    else:
                        speclogs_verifying[cpu][benchmark_size][benchmark_name] = "Error"     
            except FileNotFoundError:
                speclogs_verifying[cpu][benchmark_size][benchmark_name] = "No SPEC logs"        

### Generate the Appendix I. Working SPEC 2006 Benchmarks x CPU Model table

In [29]:
n_cols = sum([len(benchmark_sizes[cpu]) for cpu in cpus])

headers = []
dashes = []

benchmark_col_length = max([len(benchmark_name) for benchmark_name in benchmarks])
benchmark_col_length = max(benchmark_col_length, len('Benchmarks'))
benchmark_col_length = benchmark_col_length + 6

headers.append(right_padded_str(' Benchmarks', benchmark_col_length))
dashes.append('-' * len(headers[-1]))

cpu_col_length = {}

for cpu in cpus:
    for benchmark_size in benchmark_sizes[cpu]:
        col_header = ' {}/{} '.format(cpu_name_of[cpu], benchmark_size)
        cpu_col_length[(cpu, benchmark_size)] = max(len(col_header),
                                                    len('Success'),
                                                    len('Error'),
                                                    len('No SPEC logs'),
                                                    len('No results folder'))
        headers.append(right_padded_str(col_header, cpu_col_length[(cpu, benchmark_size)]))
        dashes.append('-' * cpu_col_length[(cpu, benchmark_size)])


headers = '|' + '|'.join(headers) + '|'
dashes = '|' + '|'.join(dashes) + '|'

rows = []

for benchmark_name in benchmarks:
    row = []
    row.append(right_padded_str(' {}'.format(benchmark_name), benchmark_col_length))
    for cpu in cpus:
        for benchmark_size in benchmark_sizes[cpu]:
            row.append(right_padded_str(' {}'.format(speclogs_verifying[cpu][benchmark_size][benchmark_name]),
                                        cpu_col_length[(cpu, benchmark_size)]))
    row = '|' + '|'.join(row) + '|'
    rows.append(row)
rows = '\n'.join(rows)

print(headers)
print(dashes)
print(rows)

| Benchmarks         | KVM/test        | KVM/ref         | AtomicCPU/test  | O3CPU/test      | TimingSimpleCPU/test |
|--------------------|-----------------|-----------------|-----------------|-----------------|----------------------|
| 401.bzip2          | Success         | Success         | Success         | Success         | Success              |
| 403.gcc            | Success         | Success         | Success         | Success         | Success              |
| 410.bwaves         | Success         | Success         | Success         | Success         | Success              |
| 416.gamess         | Error           | Error           | Error           | Error           | Error                |
| 429.mcf            | Success         | Success         | Success         | Success         | No SPEC logs         |
| 433.milc           | Success         | Success         | Success         | Success         | Success              |
| 434.zeusmp         | Success         | Success        

| Benchmarks         | KVM/test        | KVM/ref         | AtomicCPU/test  | O3CPU/test      | TimingSimpleCPU/test |
|--------------------|-----------------|-----------------|-----------------|-----------------|----------------------|
| 401.bzip2          | Success         | Success         | Success         | Success         | Success              |
| 403.gcc            | Success         | Success         | Success         | Success         | Success              |
| 410.bwaves         | Success         | Success         | Success         | Success         | Success              |
| 416.gamess         | Error           | Error           | Error           | Error           | Error                |
| 429.mcf            | Success         | Success         | Success         | Success         | No SPEC logs         |
| 433.milc           | Success         | Success         | Success         | Success         | Success              |
| 434.zeusmp         | Success         | Success         | Success         | No SPEC logs    | Success              |
| 435.gromacs        | Success         | Success         | Success         | Success         | Success              |
| 436.cactusADM      | Success         | Success         | Success         | Success         | Success              |
| 437.leslie3d       | Success         | Success         | Success         | Success         | Success              |
| 444.namd           | Success         | Success         | Success         | Success         | Success              |
| 445.gobmk          | Success         | Success         | Success         | No SPEC logs    | Success              |
| 453.povray         | Success         | Success         | Success         | Success         | Success              |
| 454.calculix       | Success         | Success         | Success         | Success         | Success              |
| 456.hmmer          | Success         | Success         | Success         | Success         | Success              |
| 458.sjeng          | Success         | Success         | Success         | Success         | Success              |
| 459.GemsFDTD       | Success         | Success         | Success         | Success         | Success              |
| 462.libquantum     | Success         | Success         | Success         | Success         | Success              |
| 464.h264ref        | Success         | Success         | Success         | No SPEC logs    | Success              |
| 465.tonto          | Success         | Success         | Success         | Success         | Success              |
| 470.lbm            | Success         | Success         | Success         | Success         | Success              |
| 471.omnetpp        | Success         | Success         | Success         | Success         | Success              |
| 473.astar          | Success         | Success         | Success         | No SPEC logs    | Success              |
| 481.wrf            | Error           | Error           | Error           | Error           | Error                |
| 482.sphinx3        | Success         | Success         | Success         | Success         | Success              |
| 998.specrand       | Success         | Success         | Success         | Success         | Success              |
| 999.specrand       | Success         | Success         | Success         | Success         | Success              |